## Utiliziamo LLM per risolvere problemi di Sentiment Analysis
- Utilizza il dataset 'SetFit/tweet_sentiment_extraction' presente sull'hub di HuggingFace e che ti viene fornito nelle prime celle di questo notebook (utilizza un sample di alcune tweet)
- Carica in memoria il modello microsoft/Phi-3-mini-4k-instruct da utilizzare per generare del testo (é consigliabile va bene anche caricarlo in memoria in formato 8 o 4bit)
- Scrivi delle funzioni per generare il prompt nel formato corretto specifico utilizzato dal modello. Il prompt deve essere parametrizzato e contenere informazioni sul task da svolgere e il testo su cui andare a determinare il sentiment.
- Prova a confrontare prompt con approcci diversi (Zero-Shot, Few-Shot, ...)
- Formatta l'output del modello in modo da restituire solamente il sentiment del testo
- Confronta le performance di classificazione ottenuta generando il sentiment con i diversi approcci di Prompt Engineering


<font color='red'>Va BENISSIMO fare copia-incolla dai notebook precedenti: <br>
&nbsp;&nbsp;&nbsp;&nbsp; → non devi imparare cose a memoria, devi capire come funzionano e sapere dove copiare, cosa modifiare e come! <br>
 Riuscirai a utilizzare dei modelli LLM generativi per risolvere dei task di NLP?
</font>

In [1]:
!pip install flash_attn==2.5.8
!pip install torch==2.3.1
!pip install accelerate==0.31.0
!pip install transformers==4.41.2
!pip -q install bitsandbytes accelerate xformers einops
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_c

In [2]:
import re
from sklearn.datasets import fetch_20newsgroups
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report
from datasets import load_dataset


In [3]:
device = torch.device("cuda")

In [4]:
ds = load_dataset("SetFit/tweet_sentiment_extraction")
ds_texts = ds['train']['text']
ds_targets = ds['train']['label_text']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [5]:
ds_texts[3]

' what interview! leave me alone'

In [6]:
ds_targets[3]

'negative'

In [7]:
possible_sentiments = list(set(ds_targets))
possible_sentiments

['positive', 'negative', 'neutral']